# One Way ANOVA vs Kruskal–Wallis

In this notebook i would like to introduce you a user developed function finalises the necesarry statistical steps of <b><font color="#1B4F72">One Way ANOVA or Kruskal Wallis H Test</font></b> also the related <b><font color="#1B4F72">Post Hoc Tests</font></b> and interprets the result of the hypothesis tests that are used in <b><font color="#1B4F72">One Way ANOVA and Kruskal Wallis H Test</font></b> process.

<b><font color="#1B4F72">One Way ANOVA and Kruskal Wallis H Test</font></b> method cover the statistical hypothesis tests are used to reveal significance of the difference in a metric variable values between more than two groups. There are two statistical methods to test the significance as One Way ANOVA and Kruskal Wallis H Test. According to result of <b><font color="#5DADE2">1. Step: Testing the Normality Assumption</font></b>, it is decided between two alternative methods.





<b><mark style="background-color: #1B4F72"><font color="white">*      One Way ANOVA vs Kruskal Wallis H Test         *</font></mark></b>


<b><font color="#5DADE2">1. Step: Testing the Normality Assumption</font></b>

It is necesarry to decide which way of testing the significany of difference will be choosen. Shaphiro Wilks test is a way to test if the serie has normal distribution or not. Also a Box Plot might be helpfull to observe if the serie has outliers or not.
According to result of the test it can be decided between One Way ANOVA and Kruskal Wallis H Test.

If p value of the test is greater than .05 for each group, the difference testing method will be One Way ANOVA.

In addition One Way ANOVA requires another assumption which is Homogeneity, so if the result is normal and One Way ANOVA test resulted as significant differnce occured then you can follow <b><font color="#5DADE2">3. Step: Testing the Homogeneity Assumption</font></b> to decide which one of the <b><font color="#5DADE2">4. Step: Post Hoc Tests</font></b> to use in finding out the group causing the occurrence of the difference.


If p value of the test is less than .05 for at least one group, the difference testing method will be Kruskal Wallis H Test.

In addition Kruskal Wallis H Test does not require any other assumptions, so if the result is not normal then you can jump to <b><font color="#5DADE2">2. Step: Alternative 2 -> Kruskal Wallis H Test</font></b>.

<b><font color="#5DADE2">2. Step: Alternative 1 -> One Way ANOVA</font></b>


It can be defined as an approach that tests to statistically reveal whether there is a significant difference between the means of more than two groups of data.


<b><font color="#5DADE2">2. Step: Alternative 2 -> Kruskal Wallis H Test</font></b>

It can be defined as an approach that tests to statistically reveal whether there is a significant difference between the distributions of more than two groups of data.

<b><font color="#5DADE2">3. Step: Testing the Homogeneity Assumption</font></b>

This assumption is only relevant in the case of normal distribution. It is necesarry to decide which one of the <b><font color="#1B4F72">Post Hoc Tests</font></b> to use in finding out the group causing the occurrence of the difference after One Way ANOVA result.

If p value of the test is greater than .05 <b><font color="#1B4F72">Tukey Test</font></b> will be done to find out the root of difference between groups.

If p value of the test is less than .05 <b><font color="#1B4F72">Tamhane T2 Test</font></b> will be done to find out the root of difference between groups.


## <font color="#1ABC9C">1. Background</font>

#### <font color="#1ABC9C">Aim</font> : In this study is it aimed to reveal whether there is a significant difference in the online retail variables recorded in different countries.

#### <font color="#1ABC9C">Dataset</font> : Dataset contains 2 different metric variables observed during 2010. Also another variable is created as a feature engineering example. Descriptions of the variables are reflected in the table below.

<font color="#1B4F72">No</font>|<font color="#5DADE2"> Variables </font>|<font color="#1B4F72"> Variable Type </font>|<font color="#5DADE2"> Scale Types of Data </font>
--|----------- |:-------------:|:-------------------:
1 |Quantity | Quantitative | Ratio
2 |Price | Quantitative | Ratio
3 |Total_Price | Quantitative | Ratio

## <font color="#1ABC9C">2. Data Manipulation and Feature Engineering </font>

In [ ]:
#Definition of the Necesarry Modules
import statsmodels.api as sm
from statsmodels.formula.api import ols
import scipy as stats
from scipy.stats import f_oneway
import statistics
import scipy.stats as ss
import pandas as pd
from scipy.stats import shapiro
from scipy.stats import levene
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from termcolor import colored
import random
import patsy
import plotly.figure_factory as ff
!pip install bioinfokit
!pip install scikit-posthocs
import scikit_posthocs as sp
from bioinfokit.analys import stat
from plotly.offline import init_notebook_mode, iplot, plot
import plotly as py
init_notebook_mode(connected=True)
import plotly.graph_objs as go
import warnings            
warnings.filterwarnings("ignore") 
plt.style.use('ggplot') 

#Displaying all columns and rows
pd.set_option('display.max_columns', None); pd.set_option('display.max_rows', None);

#Defining the Decimals
pd.set_option('display.float_format', lambda x: '%.2f' % x)
import matplotlib.pyplot as plt

#Importing the Datasets: Two sets for different time lines are stored in different sheets of excel
Datax = pd.read_excel("../input/online-retail-ii-data-set-from-ml-repository/online_retail_II.xlsx", sheet_name = "Year 2010-2011")

#Feature Engineering: Total Price
Datax["Total Price"] = Datax["Quantity"]*Datax["Price"]

#Cleaning Returned ("C" Coded Invioce) Items
Datax = Datax[~Datax["Invoice"].str.contains("C", na = False)]

#Dealing with DateTime Features in Pandas, Strip Year Values from Whole Date Variables
Datax["Date"] = ""
Datax["Year"] = ""
Datax["Date"] = pd.to_datetime(Datax["InvoiceDate"], format="%d/%m/%y %H:%M:%S")
Datax["Year"] = Datax["Date"].dt.year

#Creating Returned and Returned Quantity Categorical Variables
Datax["Returned"] = ""
Datax.loc[Datax["Invoice"].str.contains("C", na = False) , "Returned"] = "Returned"

#Necesarry Replacement for Creating Data Frames for the Data of Countries
Datax["Country"] = Datax["Country"].str.replace(" ","_")
Datax.rename(columns={"Total Price": "Total_Price"}, inplace = True)

#Cleaning the Missing Data
Datax = Datax[Datax.Year == 2010]
Datax.dropna(subset=["Customer ID"], how="all", inplace = True)

#Cleaning the Observations Based on a Condition
counts = Datax["Country"].value_counts()
Datax = Datax.loc[Datax["Country"].isin(counts.index[counts > 100])]

#Frequencies of Countries
Datax["Country"].value_counts()

## <font color="#1ABC9C">3. One Way ANOVA/Kruskal–Wallis in Practice</font>

In [ ]:
variables = ["Quantity","Price","Total_Price"]

for x in variables:
    
    #Convertion of Final Dataset to Necessary Format    
    CountryGroups = Datax.groupby("Country")
    Data1 = pd.concat([pd.Series(v[x].tolist(), name=k) for k, v in CountryGroups], axis=1)    
    
        #Managing Group Names as a List
    Group = []
    ln = len(Data1.columns)
    lnm = range(ln)
    for i in lnm:
        col= Data1.columns[i]
        Group.append(col)
    
    Data = pd.melt(Data1.reset_index(), id_vars=["index"],value_vars=Group)
    Data.columns = ["Index", "Country", x]
    
    #Setting the Dataframe Name
    Data1.name = str(x)
    y =  Data[x]
    z =  Data["Country"].unique()

    #Summary Statistics
    print(colored(" Difference Testing Between More Than 2 Groups for ","cyan", attrs=["bold",'reverse', 'blink'])+colored(x+" ","cyan", attrs=["bold",'reverse', 'blink']),"\n")
    print(colored("Summary Statistics by Groups for "+x,"cyan", attrs=["bold"]),"\n")
    result3 = Data.groupby("Country")[x].aggregate(["count","mean","std","median","min",max])
    result4 = result3.transpose()
    print(x)
    print(result4,"\n")
    
    print(colored("Graphs by Groups for "+x,"cyan", attrs=["bold"]),"\n")
    #Boxplots
    color_provider3 = [] #Necesarry for Historgrams
    fig = go.Figure()
    for i in Group:

        #Creating Random Colors for Each Box Plot
        r = random.randrange(0,255)
        b = random.randrange(0,255)
        g = random.randrange(0,255)
        color_provider = (r, g, b)
        color_provider2 = "rgb"+str(color_provider)
        color_provider3.append(color_provider2) #Necesarry for Historgrams
        
        fig.add_trace(go.Box(y=Data1[i], name=str(i),
                        marker_color = color_provider2))

    fig.update_layout(title_text="Boxplots for "+str(Data1.name))
    fig.show()

    #Histograms
    hist_data = []
    for i in Data1.columns:
        b = Data1[i].dropna()
        hist_data.append(b)
    fig = ff.create_distplot(hist_data, Group, bin_size=.9,
                             curve_type='normal', # override default 'kde'
                             colors=color_provider3)
    fig.update_layout(title_text="Distplot with Normal Distribution for "+str(Data1.name))
    fig.show()
       
    
    def normald():
        
        print(colored("1. Step: Testing the Normality Assumption for "+Data1.name,"cyan", attrs=["bold"]),"\n")
        Groups_shaphiro_p= []
        for i in Data1.columns:
            Group_shaphiro_w, Group_shaphiro_p = shapiro(Data1[i])
            Groups_shaphiro_p.append(Group_shaphiro_p)
        print("Shaphiro Wilk Test results in p value = ",Groups_shaphiro_p)

        if all(i > 0.05 for i in Groups_shaphiro_p): 
            print("Shaphiro Wilk Test resulted as p > .05 for all of the Groups, which indicates that H0 cannot be rejected for all of the Groups. " 
                  "Accordingly distribution of "+Data1.name+" values in all of the Groups is likely to normal distribution.","\n")
            return 1
        else:
            print("Shaphiro Wilk Test resulted as p < .05 for at least one of the Groups, which indicates that H0 is rejected for at least one of the Groups. " 
                  "Accordingly distribution of "+Data1.name+" values at least in one of the Groups is not likely to normal distribution.","\n")
            return 2

    result = normald()
       
    if result == 1:
        print(colored("2. Step: Alternative 1 -> One Way ANOVA for "+Data1.name,"cyan", attrs=["bold"]),"\n")
        model = ols("y ~ C(Country)", data=Data).fit()
        ow_anova_table = sm.stats.anova_lm(model, typ=2)
        print(ow_anova_table)     
        ow_anova_pvaule = ow_anova_table["PR(>F)"][0]
        
        if ow_anova_pvaule < 0.05:
            print("One Way ANOVA Test resulted as p < .05 which indicates that H0 is rejected."
                  " Accordingly there is a significant difference between the mean of at least one Group compared to other Groups in "+Data1.name+" values."
                  " So, Levene's Homegeneity test should be executed.","\n")
            print(colored("3. Step: Testing the Homogeneity Assumption for "+Data1.name,"cyan", attrs=["bold"]),"\n")
            res = stat()
            res.levene(df=Datax, res_var=x, xfac_var="Country")
            levene_table = res.levene_summary
            print(levene_table)
            levene_table_pvaule = levene_table["Value"][2]
            
            if levene_table_pvaule > 0.05:
                print("Levene's Homogenity Test resulted as p > .05 which indicates that H0 cannot rejected."
                      " Accordingly there is no significant difference between the variance of Groups in "+Data1.name+" values."
                      " So, Tukey test can be executed.","\n")
                print(colored("4. Step: Alternative 1 -> Post Hoc Tests-Tukey for "+Data1.name,"cyan", attrs=["bold"]),"\n")
                tukey_table = sp.posthoc_tukey(Datax, val_col=x, group_col="Country")
                print(tukey_table)
            else:
                print("Levene's Homogenity Test resulted as p < .05 which indicates that H0 is rejected."
                      " Accordingly there is a significant difference between the variance of at least one Group compared to other Groups in "+Data1.name+" values."
                      " So, Tamhane T2 test can be executed.","\n")
                print(colored("4. Step: Alternative 2 -> Post Hoc Tests-Tamhane T2 for "+Data1.name,"cyan", attrs=["bold"]),"\n")
                tamhane_table = sp.posthoc_tamhane(Datax, val_col=x, group_col="Country")
                print(tamhane_table)       
        else:
            print("One Way ANOVA Test resulted as p > .05 which indicates that H0 cannot be rejected."
                  " Accordingly there is no significant difference between the means of Groups in "+Data1.name+" values.","\n")
    else:
        print(colored("2. Step: Alternative 2 -> Kruskal-Wallis H test for "+Data1.name,"cyan", attrs=["bold"]),"\n")
        Data2 = [Datax.loc[ids,x].values for ids in Datax.groupby("Country").groups.values()]
        kruskal_hvalue, kruskal_pvaule = ss.kruskal(*Data2)
        print("Kruskal-Wallis H Test results in p value = ",kruskal_pvaule)
        if kruskal_pvaule < 0.05:
            print("Kruskal-Wallis H test resulted as p < .05 which indicates that H0 is rejected."
                  " Accordingly there is a significant difference between the distribution of at least one Group compared to other Groups in "+Data1.name+" values.","\n")
        
        else:
            print("Kruskal-Wallis H test resulted as p > .05 which indicates that H0 cannot be rejected."
                  " Accordingly there is no significant difference between the distributions of Groups in "+Data1.name+" values.","\n")          